## Finetune ResNet

In [87]:
# setting up stuff
import numpy as np

from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Activation, Dropout, Flatten, Dense

In [88]:
import keras
keras.__version__

'2.0.2'

In [89]:
# create the base pre-trained model
model = VGG16()

In [90]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [91]:
model.layers.pop()

In [92]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [93]:
model.layers

In [94]:
x = model.get_layer('fc2').output

# x = Flatten()(x)
predictions = Dense(4)(x)

In [95]:
# this is the model we will train
newmodel = Model(input=model.input, output=predictions)

/home/shubham/.pyenv/versions/anaconda3-4.1.1/lib/python3.5/site-packages/ipykernel/__main__.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  from ipykernel import kernelapp as app


In [96]:
newmodel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [97]:
for layer in newmodel.layers:
    layer.trainable = False

In [98]:
import os
import re

In [99]:
arr = []
base = 'VOCdevkit/VOC2006/Annotations/'
for file in os.listdir(base):
    with open(base + file) as f:
        z = f.read()
        img = z.split('\n')[2].split()[-1][1:-1]
        z = z.split('\n')[14].split()[-5:]
        k = z[0][1:][:-1], z[1][:-1], z[3][1:][:-1], z[4][:-1]
        k = list(map(int, k))
        arr.append((img,k))

In [100]:
import numpy as np
from scipy import misc

In [101]:
x = []
y = []
for el in arr:
    img = misc.imread(os.path.join('VOCdevkit', el[0]), flatten=False, mode=None)
    reImg = misc.imresize(img, (224,224,3))

    x.append(reImg)
    y.append(el[1])
shape = [-1, 224, 224, 3]
x = np.concatenate(x).reshape(shape)

In [102]:
x.shape

(2618, 224, 224, 3)

In [103]:
newmodel.compile(optimizer='rmsprop', loss='mean_squared_error')

In [ ]:
model.fit?

In [ ]:
newmodel.fit(x, y, epochs=10, batch_size=16)

Epoch 1/10
 960/2618 [==========>...................] - ETA: 1593s - loss: 90747.5923